In [1]:
# Input 
import pandas as pd
import numpy as np
import bert
import os
from sklearn.model_selection import train_test_split
import tensorflow
from bert import bert_tokenization
from tensorflow.keras.optimizers import SGD

max_seq_length = 256
adapter_size = 64

In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]


train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
model_name = "uncased_L-12_H-768_A-12"
model_dir = bert.fetch_google_bert_model(model_name, ".models")
model_ckpt = os.path.join(model_dir, "bert_model.ckpt")

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")
max_seq_len = 100
l_input_ids = tensorflow.keras.layers.Input(shape=(1,), dtype='int32')

model_dir + "\ vocab.txt"

Already  fetched:  uncased_L-12_H-768_A-12.zip
already unpacked at: .models\uncased_L-12_H-768_A-12


'.models\\uncased_L-12_H-768_A-12\\uncased_L-12_H-768_A-12\\ vocab.txt'

In [4]:
tokenizer = bert_tokenization.FullTokenizer(vocab_file=model_dir + "/vocab.txt")

tokenised_train_samples = []

for sample in train_samples:
    tokenised_train_samples.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sample)))
    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sample))

max_length = max(map(len, tokenised_train_samples))
max_length = 5625

train_x = []

for sample in tokenised_train_samples:
    sample = np.array(sample)
    if len(sample) < max_length:
        values_to_add = max_length - len(sample)
        train_x.append(np.pad(sample,(0,values_to_add),'constant'))
    else:
        train_x.append(sample)


tokenizer = bert_tokenization.FullTokenizer(vocab_file=model_dir + "/vocab.txt")

tokenised_test_samples = []

for sample in test_samples:
    tokenised_test_samples.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sample)))
    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sample))

max_length = max(map(len, tokenised_train_samples))
max_length = 5625

test_x = []

for sample in tokenised_test_samples:
    sample = np.array(sample)
    if len(sample) < max_length:
        values_to_add = max_length - len(sample)
        test_x.append(np.pad(sample,(0,values_to_add),'constant'))
    else:
        test_x.append(sample)

In [5]:
bertLayer = l_bert(l_input_ids)
x = tensorflow.keras.layers.Conv1D(128, 1, activation="relu")(bertLayer)
x = tensorflow.keras.layers.MaxPooling1D(1)(x)
x = tensorflow.keras.layers.Conv1D(128, 1, activation="relu")(x)
x = tensorflow.keras.layers.MaxPooling1D(1)(x)
x = tensorflow.keras.layers.Conv1D(128, 1, activation="relu")(x)
x = tensorflow.keras.layers.GlobalMaxPooling1D()(x)
x = tensorflow.keras.layers.Dense(128, activation="relu")(x)
x = tensorflow.keras.layers.Dropout(0.5)(x)
preds = tensorflow.keras.layers.Dense(units=1)(x)
cnn_small = tensorflow.keras.Model(l_input_ids, preds)
cnn_small.summary()
cnn_small.compile(optimizer=SGD(learning_rate=0.01),loss='mean_absolute_error',metrics=[tensorflow.keras.metrics.MeanSquaredError(),tensorflow.keras.losses.MeanAbsoluteError(),tensorflow.keras.losses.MeanAbsolutePercentageError()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
bert (BertModelLayer)        (None, None, 768)         108890112 
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         98432     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         16512 

In [6]:
bertLayer = l_bert(l_input_ids)
x = tensorflow.keras.layers.Conv1D(256, 1, activation="relu")(bertLayer)
x = tensorflow.keras.layers.MaxPooling1D(1)(x)
x = tensorflow.keras.layers.Conv1D(256, 1, activation="relu")(x)
x = tensorflow.keras.layers.MaxPooling1D(1)(x)
x = tensorflow.keras.layers.Conv1D(256, 1, activation="relu")(x)
x = tensorflow.keras.layers.GlobalMaxPooling1D()(x)
x = tensorflow.keras.layers.Dense(256, activation="relu")(x)
x = tensorflow.keras.layers.Dropout(0.5)(x)
preds = tensorflow.keras.layers.Dense(units=1)(x)
cnn_large = tensorflow.keras.Model(l_input_ids, preds)
cnn_large.summary()
cnn_large.compile(optimizer=SGD(learning_rate=0.01),loss='mean_absolute_error',metrics=[tensorflow.keras.metrics.MeanSquaredError(),tensorflow.keras.losses.MeanAbsoluteError(),tensorflow.keras.losses.MeanAbsolutePercentageError()])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
bert (BertModelLayer)        (None, None, 768)         108890112 
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 256)         196864    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 256)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 256)         65792     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 256)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 256)         6579

In [7]:
results = pd.DataFrame()

In [8]:
#100s
cnn_small.fit(x=train_x,y=train_labels,batch_size=10,epochs=100)
results["CNN Small E100 B10"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=10,epochs=100)
results["CNN Large E100 B10"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_small.fit(x=train_x,y=train_labels,batch_size=32,epochs=100)
results["CNN Small E100 B32"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=32,epochs=100)
results["CNN Large E100 B32"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

results

Train on 5625 samples
Epoch 1/100
5625/5625 [==============================] - 150s 27ms/sample - loss: 1.2288 - mean_squared_error: 2.3156 - mean_absolute_error: 1.2281 - mean_absolute_percentage_error: 34.4582
Epoch 2/100
5625/5625 [==============================] - 136s 24ms/sample - loss: 1.0753 - mean_squared_error: 1.7385 - mean_absolute_error: 1.0752 - mean_absolute_percentage_error: 30.7828
Epoch 3/100
5625/5625 [==============================] - 142s 25ms/sample - loss: 1.0581 - mean_squared_error: 1.6588 - mean_absolute_error: 1.0582 - mean_absolute_percentage_error: 30.8538
Epoch 4/100
5625/5625 [==============================] - 140s 25ms/sample - loss: 1.0344 - mean_squared_error: 1.5784 - mean_absolute_error: 1.0341 - mean_absolute_percentage_error: 30.2354
Epoch 5/100
5625/5625 [==============================] - 141s 25ms/sample - loss: 1.0071 - mean_squared_error: 1.4873 - mean_absolute_error: 1.0071 - mean_absolute_percentage_error: 29.7206
Epoch 6/100
5625/5625 [=====

,CNN Small E100 B10,CNN Large E100 B10,CNN Small E100 B32,CNN Large E100 B32
0,0.948128,0.948348,0.948182,0.948396
1,1.305763,1.302709,1.304047,1.306573
2,0.948282,0.948501,0.948336,0.948550
3,29.322237,29.243969,29.277596,29.349554


In [9]:
#75s
cnn_small.fit(x=train_x,y=train_labels,batch_size=10,epochs=75)
results["CNN Small E75 B10"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=10,epochs=75)
results["CNN Large E75 B10"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_small.fit(x=train_x,y=train_labels,batch_size=32,epochs=75)
results["CNN Small E75 B32"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=32,epochs=75)
results["CNN Large E75 B32"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

results

Train on 5625 samples
Epoch 1/75
5625/5625 [==============================] - 143s 26ms/sample - loss: 0.9485 - mean_squared_error: 1.3033 - mean_absolute_error: 0.9482 - mean_absolute_percentage_error: 29.2297
Epoch 2/75
5625/5625 [==============================] - 145s 26ms/sample - loss: 0.9485 - mean_squared_error: 1.3027 - mean_absolute_error: 0.9484 - mean_absolute_percentage_error: 29.2154
Epoch 3/75
5625/5625 [==============================] - 143s 25ms/sample - loss: 0.9486 - mean_squared_error: 1.3040 - mean_absolute_error: 0.9486 - mean_absolute_percentage_error: 29.2524
Epoch 4/75
5625/5625 [==============================] - 145s 26ms/sample - loss: 0.9486 - mean_squared_error: 1.3038 - mean_absolute_error: 0.9486 - mean_absolute_percentage_error: 29.2554
Epoch 5/75
5625/5625 [==============================] - 145s 26ms/sample - loss: 0.9485 - mean_squared_error: 1.3036 - mean_absolute_error: 0.9485 - mean_absolute_percentage_error: 29.2439
Epoch 6/75
5625/5625 [===========

,CNN Small E100 B10,CNN Large E100 B10,CNN Small E100 B32,CNN Large E100 B32,CNN Small E75 B10,CNN Large E75 B10,CNN Small E75 B32,CNN Large E75 B32
0,0.948128,0.948348,0.948182,0.948396,0.948162,0.949150,0.948088,0.948099
1,1.305763,1.302709,1.304047,1.306573,1.304219,1.308950,1.304838,1.304744
2,0.948282,0.948501,0.948336,0.948550,0.948315,0.949306,0.948242,0.948253
3,29.322237,29.243969,29.277596,29.349554,29.281797,29.426428,29.296791,29.294519


In [11]:
#50s
cnn_small.fit(x=train_x,y=train_labels,batch_size=10,epochs=50)
results["CNN Small E50 B10"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=10,epochs=50)
results["CNN Large E50 B10"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_small.fit(x=train_x,y=train_labels,batch_size=32,epochs=50)
results["CNN Small E50 B32"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=32,epochs=50)
results["CNN Large E50 B32"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

results

Train on 5625 samples
Epoch 1/50
5625/5625 [==============================] - 171s 30ms/sample - loss: 0.9485 - mean_squared_error: 1.3039 - mean_absolute_error: 0.9486 - mean_absolute_percentage_error: 29.2686
Epoch 2/50
5625/5625 [==============================] - 182s 32ms/sample - loss: 0.9486 - mean_squared_error: 1.3022 - mean_absolute_error: 0.9490 - mean_absolute_percentage_error: 29.2552
Epoch 3/50
5625/5625 [==============================] - 182s 32ms/sample - loss: 0.9485 - mean_squared_error: 1.3043 - mean_absolute_error: 0.9488 - mean_absolute_percentage_error: 29.3150
Epoch 4/50
5625/5625 [==============================] - 182s 32ms/sample - loss: 0.9485 - mean_squared_error: 1.3035 - mean_absolute_error: 0.9484 - mean_absolute_percentage_error: 29.2528
Epoch 5/50
5625/5625 [==============================] - 182s 32ms/sample - loss: 0.9485 - mean_squared_error: 1.3037 - mean_absolute_error: 0.9480 - mean_absolute_percentage_error: 29.2417
Epoch 6/50
5625/5625 [===========

In [1]:
results

NameError: name 'results' is not defined

In [ ]:
#20s
cnn_small.fit(x=train_x,y=train_labels,batch_size=10,epochs=20)
results["CNN Small E20 B10"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=10,epochs=20)
results["CNN Large E20 B10"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)


cnn_small.fit(x=train_x,y=train_labels,batch_size=32,epochs=20)
results["CNN Small E20 B32"] = cnn_small.evaluate(x=train_x,y=train_labels,verbose=0)

cnn_large.fit(x=train_x,y=train_labels,batch_size=32,epochs=20)
results["CNN Large E20 B32"] = cnn_large.evaluate(x=train_x,y=train_labels,verbose=0)

results

In [ ]:
results
results.to_csv("bert_initial_tests.csv")